In [951]:
import argparse
import torch
from torch.autograd import Variable
from torch import nn
from seq2seq.util.checkpoint import Checkpoint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy
from mpl_toolkits.axes_grid1 import AxesGrid

In [1012]:
class Model(object):
    def __init__(self, model):
        self.model = model
        params = {}
        for name, param in self.model.named_parameters():
            param = param.data.numpy()
            params[name]=pd.DataFrame(param)
        self.params = params
        
    def get_param_names(self):
        return [name for name, _ in self.model.named_parameters()]

    def get_modules(self):
        return [mod for mod in self.model.modules()]

    def get_param_by_name(self, name):
        return pd.DataFrame(self.params[name])

    def heatmap(self):
        return {k: sns.heatmap(v) for k, v in self.params.items()}

    def apply_mean(self):
        return {k: np.ravel(v).mean() if v.shape != (1,1) else np.NaN for k, v in self.params.items()}
       
    def apply_std(self):
        return {k: np.ravel(v).std() if v.shape != (1,1) else np.NaN for k, v in self.params.items()}
    
    def apply_min(self):
        return {k: np.ravel(v).min() for k, v in self.params.items()}
    
    def apply_max(self):
        return {k: np.ravel(v).max() for k, v in self.params.items()}
        
    def apply_norm(self):
        return {k: np.linalg.norm(np.ravel(v)) if v.shape != (1,1) else np.NaN for k, v in self.params.items()}
            
    def param_to_dist(self,name):
        data = self.params[name]
        data = np.ravel(data)
#         # best fit of data
#         (mu, sigma) = norm.fit(data)

#         # the histogram of the data
#         n, bins, patches = plt.hist(data, 20, normed=1)

#         # add a 'best fit' line
#         y = mlab.normpdf( bins, mu, sigma)
#         l = plt.plot(bins, y, 'r--', linewidth=2)
#        return scipy.stats.norm(mu, sigma)
        hist, _ = np.histogram(data, bins=50, range=[-1, 1], density=True)
        return hist
        

            
class Models(object):
    def mean_of(self, data):
        one_key = list(data.keys())[0]
        return {param: np.mean([data[name][param] for name in data.keys()]) for param in data[one_key].keys()}
    
    def load_models(self):
        models = {}
        files = os.listdir(self.model_path)
        for file in files: 
            if not file.startswith('.'):
                print('loading: ', self.model_path + '/' + file)
                checkpoint = Checkpoint.load(self.model_path + '/' + file)
                seq2seq = checkpoint.model
                models[file] = Model(seq2seq)
        return models
    
    def __init__(self, model_path,title):
        
        self.image_folder = 'images/'
        self.title = title
        self.model_path = model_path
        
        self.models = self.load_models()

        ## calculate mean, std, norm
        self.means = {name: model.apply_mean() for name,model in self.models.items()}
        self.stds = {name : model.apply_std() for name,model in self.models.items()}
        self.norms = {name: model.apply_norm() for name, model in self.models.items()}
        self.mins = {name: model.apply_min() for name, model in self.models.items()}
        self.maxs = {name: model.apply_max() for name, model in self.models.items()}
        
        ## caluclate mean of means, stds, norms
        self.mean_of_means = self.mean_of(self.means)
        self.mean_of_stds = self.mean_of(self.stds)
        self.mean_of_norms = self.mean_of(self.norms) 
        self.maxs = self.mean_of(self.maxs) 
        self.mins = self.mean_of(self.mins) 
        
        # fill data into df 
        df = pd.DataFrame.from_dict(self.mean_of_means,  orient='index')
        df = df.rename(columns={0: 'mean of means'})
        df['mean of stds'] = self.mean_of_stds.values()
        df['mean of norms'] = self.mean_of_norms.values()
        df['mean of maxs'] = self.maxs.values()
        df['mean of mins'] = self.mins.values()
        self.df = df        
        
    
    def apply_heatmap(self):
        for model_name, model in self.models.items():
            for param_name in self.models[model_name].params.keys():
                plt.figure()
                sns.heatmap(model.params[param_name])
                plt.title(self.title + ' (' + model_name + ') - \n heatmap of param: ' + param_name)
                plt.savefig('images/' + self.title + '_' + model_name + '_' + param_name + '.png', dpi=300)
                plt.show()
                
    def apply_heatmap_by_name(self,param_name):
        for model_name, model in self.models.items():
            plt.figure()
            sns.heatmap(model.params[param_name])
            plt.title(self.title + ' (' + model_name + ') - \n heatmap of param: ' + param_name)
            plt.savefig(image_folder + self.title + '_' + model_name + '_' + param_name + '.png', dpi=300)
            plt.show()    

In [1013]:
guided_gru = Models('../machine-zoo/guided/gru', 'Guided_GRU')
baseline_gru = Models('../machine-zoo/baseline/gru', 'Baseline_GRU')

guided_lstm = Models('../machine-zoo/guided/lstm', 'Guided_LSTM')
baseline_lstm = Models('../machine-zoo/baseline/lstm', 'Baseline_LSTM')

loading:  ../machine-zoo/guided/gru/1
loading:  ../machine-zoo/guided/gru/2
loading:  ../machine-zoo/guided/gru/3
loading:  ../machine-zoo/guided/gru/4
loading:  ../machine-zoo/guided/gru/5
loading:  ../machine-zoo/baseline/gru/1
loading:  ../machine-zoo/baseline/gru/2
loading:  ../machine-zoo/baseline/gru/3
loading:  ../machine-zoo/baseline/gru/4
loading:  ../machine-zoo/baseline/gru/5
loading:  ../machine-zoo/guided/lstm/1
loading:  ../machine-zoo/guided/lstm/2
loading:  ../machine-zoo/guided/lstm/3
loading:  ../machine-zoo/guided/lstm/4
loading:  ../machine-zoo/guided/lstm/5
loading:  ../machine-zoo/baseline/lstm/1
loading:  ../machine-zoo/baseline/lstm/2
loading:  ../machine-zoo/baseline/lstm/3
loading:  ../machine-zoo/baseline/lstm/4
loading:  ../machine-zoo/baseline/lstm/5


### Sizes of layers

In [1014]:
for name, params in guided_lstm.models['1'].params.items():
    print(params.shape, name)

(19, 16) encoder.embedding.weight
(2048, 16) encoder.rnn.weight_ih_l0
(2048, 512) encoder.rnn.weight_hh_l0
(2048, 1) encoder.rnn.bias_ih_l0
(2048, 1) encoder.rnn.bias_hh_l0
(2048, 512) decoder.rnn.weight_ih_l0
(2048, 512) decoder.rnn.weight_hh_l0
(2048, 1) decoder.rnn.bias_ih_l0
(2048, 1) decoder.rnn.bias_hh_l0
(11, 512) decoder.embedding.weight
(512, 1024) decoder.attention.method.mlp.weight
(512, 1) decoder.attention.method.mlp.bias
(1, 512) decoder.attention.method.out.weight
(1, 1) decoder.attention.method.out.bias
(11, 512) decoder.out.weight
(11, 1) decoder.out.bias
(512, 1024) decoder.ffocus_merge.weight
(512, 1) decoder.ffocus_merge.bias


## Statistical Analysis

Guided GRU

In [1015]:
guided_gru.df

,mean of means,mean of stds,mean of norms,mean of maxs,mean of mins
encoder.embedding.weight,-0.000959,0.139349,2.429867,0.376077,-0.403524
encoder.rnn.weight_ih_l0,-0.000206,0.080538,12.626729,0.542031,-0.511061
encoder.rnn.weight_hh_l0,0.000058,0.065287,57.897675,0.412640,-0.394051
encoder.rnn.bias_ih_l0,-0.012083,0.072939,2.898563,0.176322,-0.312060
encoder.rnn.bias_hh_l0,-0.013354,0.070808,2.824670,0.194362,-0.297983
decoder.rnn.weight_ih_l0,0.000041,0.065539,58.121387,0.742477,-0.713911
decoder.rnn.weight_hh_l0,-0.000056,0.058843,52.185558,0.291974,-0.281988
decoder.rnn.bias_ih_l0,-0.025098,0.059747,2.540183,0.144791,-0.195006
decoder.rnn.bias_hh_l0,-0.025394,0.059537,2.536941,0.131834,-0.197489
decoder.embedding.weight,0.000316,0.147676,11.082689,0.395972,-0.391510


Baseline GRU

In [1016]:
baseline_gru.df

,mean of means,mean of stds,mean of norms,mean of maxs,mean of mins
encoder.embedding.weight,-1.759272e-03,0.146639,2.557299,0.466294,-0.440219
encoder.rnn.weight_ih_l0,1.795653e-03,0.132039,20.707676,0.657196,-0.636781
encoder.rnn.weight_hh_l0,2.041644e-04,0.088030,78.066849,0.566109,-0.596124
encoder.rnn.bias_ih_l0,-1.620729e-02,0.092863,3.695691,0.276811,-0.357539
encoder.rnn.bias_hh_l0,-1.591059e-02,0.092919,3.696127,0.273887,-0.335439
decoder.rnn.weight_ih_l0,-1.277724e-04,0.068788,61.002495,0.457143,-0.430959
decoder.rnn.weight_hh_l0,-1.709472e-04,0.075933,67.339149,0.425063,-0.432390
decoder.rnn.bias_ih_l0,-2.985862e-02,0.075753,3.191333,0.192314,-0.249801
decoder.rnn.bias_hh_l0,-2.878932e-02,0.075507,3.167907,0.207104,-0.249486
decoder.embedding.weight,-6.870941e-05,0.084614,6.350204,0.323575,-0.334121


Guided LSTM

In [1017]:
guided_lstm.df

,mean of means,mean of stds,mean of norms,mean of maxs,mean of mins
encoder.embedding.weight,0.002058,0.179018,3.123551,0.543385,-0.524266
encoder.rnn.weight_ih_l0,0.000245,0.113211,20.516216,0.710162,-0.710747
encoder.rnn.weight_hh_l0,0.000026,0.071164,72.872803,0.551239,-0.668254
encoder.rnn.bias_ih_l0,-0.010346,0.082105,3.746127,0.408405,-0.251434
encoder.rnn.bias_hh_l0,-0.010515,0.081048,3.700511,0.403438,-0.238568
decoder.rnn.weight_ih_l0,0.000095,0.080769,82.708237,0.941351,-0.865073
decoder.rnn.weight_hh_l0,-0.000130,0.073191,74.948914,0.402468,-0.391413
decoder.rnn.bias_ih_l0,-0.024615,0.059836,2.930603,0.152430,-0.214242
decoder.rnn.bias_hh_l0,-0.024132,0.060696,2.957357,0.167402,-0.216230
decoder.embedding.weight,0.000231,0.179740,13.489433,0.491869,-0.492699


Baseline LSTM

In [1018]:
baseline_lstm.df

,mean of means,mean of stds,mean of norms,mean of maxs,mean of mins
encoder.embedding.weight,-0.001442,0.162086,2.826329,0.492238,-0.502056
encoder.rnn.weight_ih_l0,0.000913,0.146842,26.588650,0.684907,-0.707094
encoder.rnn.weight_hh_l0,-0.000176,0.089515,91.664444,0.570511,-0.610069
encoder.rnn.bias_ih_l0,-0.037526,0.090355,4.483636,0.339953,-0.328324
encoder.rnn.bias_hh_l0,-0.038465,0.090417,4.501397,0.307354,-0.318395
decoder.rnn.weight_ih_l0,-0.000139,0.080955,82.899498,0.517021,-0.515518
decoder.rnn.weight_hh_l0,-0.000294,0.084335,86.370056,0.489893,-0.495761
decoder.rnn.bias_ih_l0,-0.035166,0.065420,3.369680,0.178394,-0.243665
decoder.rnn.bias_hh_l0,-0.033474,0.065755,3.347765,0.171613,-0.239341
decoder.embedding.weight,-0.000122,0.132717,9.960362,0.474859,-0.472432


## Analysis of Distribution

GRU Guided vs. Baseline

In [1042]:
class Analysis(object):
    
    def return_intersection(self, hist_1, hist_2):
        minima = np.minimum(hist_1, hist_2)
        intersection = np.true_divide(np.sum(minima), np.sum(hist_2))
        return intersection

    def KL(self,dist_1, dist_2):
        x = np.linspace(-1, 1, 100)
        return scipy.stats.entropy(dist_1.pdf(x),dist_2.pdf(x))  
    
    def compare_distributions(self):
        dist = {}
        for model_name_A in self.models_A.keys():
            for model_name_B in self.models_B.keys():
                per_model = {}
                for param in self.models_A[list(self.models_A.keys())[0]].params.keys():
                    if not self.models_A[model_name_A].params[param].shape == (1,1):
                        dist_1 = self.models_A[model_name_A].param_to_dist(param)
                        dist_2 = self.models_B[model_name_B].param_to_dist(param)
                        per_model[param] = self.return_intersection(dist_1, dist_2)
                key = model_name_A + '_' + model_name_B
                dist[key] = per_model
        return pd.DataFrame.from_dict(dist, orient='index')
    
    def compare_dist_within(self, models):
        dist = {}
        for param in models[list(models.keys())[0]].params.keys():
            within_model = {}
            for model_name_A in models.keys():
                for model_name_B in models.keys():
                    if (model_name_A != model_name_B) and (not models[model_name_A].params[param].shape == (1,1)):
                        dist_1 = models[model_name_A].param_to_dist(param)
                        dist_2 = models[model_name_B].param_to_dist(param)
                        key = model_name_A + '_' + model_name_B
                        within_model[key] = self.return_intersection(dist_1, dist_2)
                dist[param] = within_model
        return pd.DataFrame.from_dict(dist, orient='index')
    
    def __init__(self, models_A, models_B):
        self.models_A = models_A.models
        self.models_B = models_B.models
        self.dist = self.compare_distributions()
        self.dist_within_A = self.compare_dist_within(self.models_A)
        self.dist_within_B = self.compare_dist_within(self.models_B)
        
    def compare_dist_by_name(self,param):
        for model_name in self.models_A.keys():
            assert(self.models_A[model_name].params[param].shape != (1,1))
            dist_1 = self.models_A[model_name].param_to_dist(param)
            dist_2 = self.models_B[model_name].param_to_dist(param)
            print(self.return_intersection(dist_1, dist_2))

In [1043]:
analysis_GRU = Analysis(baseline_gru, guided_gru)

In [1044]:
analysis_GRU.dist.describe()

,encoder.embedding.weight,encoder.rnn.weight_ih_l0,encoder.rnn.weight_hh_l0,encoder.rnn.bias_ih_l0,encoder.rnn.bias_hh_l0,decoder.rnn.weight_ih_l0,decoder.rnn.weight_hh_l0,decoder.rnn.bias_ih_l0,decoder.rnn.bias_hh_l0,decoder.embedding.weight,decoder.attention.method.mlp.weight,decoder.attention.method.mlp.bias,decoder.attention.method.out.weight,decoder.out.weight,decoder.out.bias,decoder.ffocus_merge.weight,decoder.ffocus_merge.bias
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.868816,0.744264,0.864755,0.875599,0.872526,0.933552,0.892613,0.893203,0.893776,0.566214,0.941055,0.874766,0.565773,0.918111,0.592727,0.956731,0.786563
std,0.016620,0.019523,0.019201,0.031462,0.034271,0.012509,0.015916,0.016076,0.022599,0.032272,0.017775,0.024357,0.116134,0.015593,0.141616,0.011778,0.034763
min,0.845395,0.703044,0.826725,0.827474,0.830078,0.912841,0.861463,0.867839,0.848307,0.517578,0.910160,0.824219,0.365234,0.891335,0.363636,0.936085,0.730469
25%,0.858553,0.732178,0.854861,0.847005,0.847656,0.923400,0.880046,0.882812,0.878255,0.540661,0.933981,0.859375,0.474609,0.908913,0.454545,0.947453,0.765625
50%,0.865132,0.744303,0.865724,0.881510,0.863281,0.931273,0.892869,0.891927,0.893229,0.562500,0.938148,0.876953,0.582031,0.919212,0.545455,0.955082,0.777344
75%,0.881579,0.757406,0.875916,0.890625,0.882161,0.945811,0.905159,0.902995,0.910156,0.586470,0.948311,0.888672,0.651218,0.930220,0.727273,0.967932,0.818359
max,0.904605,0.776123,0.903155,0.927734,0.940104,0.953189,0.923272,0.929036,0.934245,0.640270,0.974909,0.919922,0.757813,0.945845,0.818182,0.973284,0.861328


GRU distribution within Baseline

In [1045]:
analysis_GRU.dist_within_A.T.describe()

,decoder.attention.method.mlp.bias,decoder.attention.method.mlp.weight,decoder.attention.method.out.weight,decoder.embedding.weight,decoder.ffocus_merge.bias,decoder.ffocus_merge.weight,decoder.out.bias,decoder.out.weight,decoder.rnn.bias_hh_l0,decoder.rnn.bias_ih_l0,decoder.rnn.weight_hh_l0,decoder.rnn.weight_ih_l0,encoder.embedding.weight,encoder.rnn.bias_hh_l0,encoder.rnn.bias_ih_l0,encoder.rnn.weight_hh_l0,encoder.rnn.weight_ih_l0
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.904102,0.976657,0.793750,0.937891,0.918555,0.978475,0.563636,0.961310,0.949349,0.955924,0.987535,0.982306,0.877632,0.934635,0.933138,0.977396,0.970605
std,0.029293,0.016048,0.051785,0.032583,0.031710,0.012559,0.143285,0.011808,0.011599,0.010687,0.006555,0.009659,0.030740,0.020181,0.015986,0.012826,0.010356
min,0.853516,0.944164,0.722656,0.879261,0.859375,0.963852,0.454545,0.941584,0.930990,0.936849,0.975995,0.968484,0.832237,0.901042,0.910807,0.951706,0.958618
25%,0.880859,0.968166,0.761719,0.912642,0.902344,0.966871,0.454545,0.952770,0.941406,0.944010,0.982014,0.976097,0.845395,0.927734,0.920573,0.969648,0.959595
50%,0.905273,0.974147,0.784180,0.945401,0.924805,0.975695,0.454545,0.962979,0.951172,0.958984,0.988106,0.979717,0.878289,0.935872,0.933594,0.980388,0.970703
75%,0.925781,0.996176,0.804687,0.966264,0.943359,0.990767,0.727273,0.971236,0.952474,0.962891,0.993974,0.991597,0.907895,0.947266,0.941406,0.988794,0.979533
max,0.943359,0.997530,0.912109,0.973544,0.966797,0.996452,0.818182,0.979581,0.970703,0.972005,0.995593,0.995433,0.917763,0.962891,0.961589,0.993234,0.989502


GRU distribution within Guided

In [1046]:
analysis_GRU.dist_within_B.T.describe()

,decoder.attention.method.mlp.bias,decoder.attention.method.mlp.weight,decoder.attention.method.out.weight,decoder.embedding.weight,decoder.ffocus_merge.bias,decoder.ffocus_merge.weight,decoder.out.bias,decoder.out.weight,decoder.rnn.bias_hh_l0,decoder.rnn.bias_ih_l0,decoder.rnn.weight_hh_l0,decoder.rnn.weight_ih_l0,encoder.embedding.weight,encoder.rnn.bias_hh_l0,encoder.rnn.bias_ih_l0,encoder.rnn.weight_hh_l0,encoder.rnn.weight_ih_l0
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.949219,0.990875,0.824148,0.950284,0.931641,0.991665,0.609091,0.975568,0.973763,0.962630,0.981162,0.993493,0.876974,0.963932,0.959180,0.987489,0.978223
std,0.008549,0.003208,0.049207,0.014041,0.017051,0.003653,0.177214,0.005024,0.008066,0.007808,0.008861,0.002736,0.017679,0.010592,0.010068,0.008517,0.007818
min,0.935547,0.985191,0.738281,0.928622,0.898437,0.983929,0.363636,0.970170,0.957031,0.947266,0.962195,0.988605,0.848684,0.943359,0.943359,0.971859,0.963379
25%,0.943359,0.989058,0.791016,0.939986,0.921875,0.989885,0.454545,0.970526,0.971354,0.957031,0.974440,0.991144,0.865132,0.961589,0.953776,0.980502,0.972005
50%,0.949219,0.991209,0.841223,0.951083,0.933594,0.992360,0.545455,0.974964,0.973307,0.963216,0.985835,0.993839,0.878289,0.966146,0.957682,0.990720,0.980815
75%,0.953125,0.993398,0.855469,0.960405,0.943359,0.994270,0.818182,0.979403,0.979818,0.970703,0.987755,0.995491,0.888158,0.970703,0.968099,0.994704,0.984701
max,0.966797,0.995244,0.886121,0.969638,0.957031,0.996153,0.909091,0.985263,0.986328,0.972005,0.991168,0.996934,0.907895,0.977214,0.973958,0.998002,0.988078


LSTM Guided vs. Baseline

In [1047]:
analysis_LSTM = Analysis(guided_lstm, baseline_lstm)

In [1048]:
analysis_LSTM.dist.describe()

,encoder.embedding.weight,encoder.rnn.weight_ih_l0,encoder.rnn.weight_hh_l0,encoder.rnn.bias_ih_l0,encoder.rnn.bias_hh_l0,decoder.rnn.weight_ih_l0,decoder.rnn.weight_hh_l0,decoder.rnn.bias_ih_l0,decoder.rnn.bias_hh_l0,decoder.embedding.weight,decoder.attention.method.mlp.weight,decoder.attention.method.mlp.bias,decoder.attention.method.out.weight,decoder.out.weight,decoder.out.bias,decoder.ffocus_merge.weight,decoder.ffocus_merge.bias
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.844079,0.844094,0.898595,0.874434,0.868320,0.951818,0.938035,0.910605,0.917969,0.661364,0.942557,0.845234,0.636315,0.887791,0.549091,0.945640,0.860391
std,0.028249,0.090856,0.064630,0.092074,0.096388,0.037297,0.052990,0.028361,0.029646,0.078769,0.053289,0.042907,0.132852,0.033851,0.124427,0.019945,0.036527
min,0.782895,0.652130,0.776758,0.683105,0.677246,0.869842,0.824250,0.864746,0.854980,0.526456,0.830433,0.755859,0.386719,0.832564,0.363636,0.900955,0.787109
25%,0.822368,0.768921,0.862494,0.857910,0.841797,0.935133,0.936102,0.889648,0.891602,0.608132,0.918846,0.822266,0.590539,0.863814,0.454545,0.935379,0.833984
50%,0.851974,0.858276,0.906240,0.910156,0.905762,0.969708,0.953884,0.913086,0.929199,0.655540,0.969986,0.851562,0.649133,0.881570,0.545455,0.947441,0.869141
75%,0.868421,0.913635,0.938403,0.935547,0.943359,0.977510,0.976146,0.927734,0.937012,0.702770,0.983482,0.880859,0.744301,0.911932,0.636364,0.963771,0.880859
max,0.878289,0.968445,0.995316,0.966309,0.965820,0.991642,0.995377,0.957520,0.961914,0.815341,0.991564,0.902344,0.822266,0.956143,0.818182,0.971615,0.923828


LSTM distribution within Baseline

In [1040]:
analysis_LSTM.dist_within_A.T.describe()

,decoder.attention.method.mlp.bias,decoder.attention.method.mlp.weight,decoder.attention.method.out.weight,decoder.embedding.weight,decoder.ffocus_merge.bias,decoder.ffocus_merge.weight,decoder.out.bias,decoder.out.weight,decoder.rnn.bias_hh_l0,decoder.rnn.bias_ih_l0,decoder.rnn.weight_hh_l0,decoder.rnn.weight_ih_l0,encoder.embedding.weight,encoder.rnn.bias_hh_l0,encoder.rnn.bias_ih_l0,encoder.rnn.weight_hh_l0,encoder.rnn.weight_ih_l0
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.926172,0.982899,0.853680,0.923828,0.914062,0.979446,0.554545,0.949911,0.955957,0.953662,0.980909,0.978113,0.845066,0.944678,0.950439,0.978631,0.931378
std,0.023976,0.008314,0.027138,0.024637,0.014561,0.010835,0.105935,0.011467,0.012536,0.015582,0.011446,0.013330,0.031059,0.011881,0.015049,0.011058,0.026959
min,0.884766,0.965364,0.793546,0.875355,0.896484,0.957707,0.454545,0.930930,0.935547,0.929688,0.959282,0.954600,0.799342,0.924316,0.915527,0.956105,0.879486
25%,0.904297,0.978104,0.848589,0.901278,0.902344,0.972893,0.454545,0.940163,0.945312,0.941406,0.977062,0.964038,0.822368,0.934570,0.946289,0.970917,0.917786
50%,0.926758,0.985091,0.856036,0.929066,0.908203,0.981558,0.545455,0.949396,0.956299,0.948975,0.981562,0.980787,0.847039,0.945068,0.953857,0.983753,0.938431
75%,0.941406,0.989031,0.867187,0.937855,0.929687,0.987520,0.636364,0.957031,0.966309,0.967773,0.988399,0.990439,0.858553,0.951660,0.958496,0.986762,0.954315
max,0.962891,0.992342,0.894202,0.956143,0.937500,0.992867,0.727273,0.967685,0.976074,0.975586,0.997905,0.995173,0.914474,0.964844,0.971680,0.992181,0.964478


LSTM distribution within Guided

In [1050]:
analysis_LSTM.dist_within_B.T.describe()

,decoder.attention.method.mlp.bias,decoder.attention.method.mlp.weight,decoder.attention.method.out.weight,decoder.embedding.weight,decoder.ffocus_merge.bias,decoder.ffocus_merge.weight,decoder.out.bias,decoder.out.weight,decoder.rnn.bias_hh_l0,decoder.rnn.bias_ih_l0,decoder.rnn.weight_hh_l0,decoder.rnn.weight_ih_l0,encoder.embedding.weight,encoder.rnn.bias_hh_l0,encoder.rnn.bias_ih_l0,encoder.rnn.weight_hh_l0,encoder.rnn.weight_ih_l0
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.898242,0.929037,0.766406,0.836115,0.882227,0.935238,0.527273,0.894283,0.927881,0.922803,0.931231,0.937875,0.839803,0.838428,0.851855,0.909964,0.846725
std,0.026712,0.052292,0.099526,0.074627,0.030732,0.036078,0.130579,0.066375,0.034534,0.030875,0.051783,0.039479,0.023819,0.096397,0.096680,0.049387,0.079452
min,0.853516,0.856874,0.591797,0.689453,0.828125,0.867384,0.363636,0.785334,0.878906,0.868652,0.837410,0.868717,0.786184,0.685547,0.696289,0.804965,0.682556
25%,0.880859,0.863214,0.736328,0.763494,0.876953,0.909126,0.454545,0.818892,0.891113,0.898926,0.888148,0.897515,0.828947,0.741211,0.747559,0.874923,0.788300
50%,0.890625,0.934115,0.782227,0.867898,0.886719,0.947485,0.500000,0.929510,0.940186,0.918701,0.948925,0.945325,0.840461,0.881592,0.901367,0.919159,0.882492
75%,0.921875,0.991573,0.845703,0.901811,0.910156,0.958042,0.636364,0.952770,0.958008,0.956055,0.981480,0.971201,0.851974,0.924805,0.931641,0.940362,0.897339
max,0.943359,0.992342,0.882812,0.907138,0.917969,0.986622,0.727273,0.961648,0.972656,0.963379,0.996182,0.991217,0.871711,0.936523,0.954102,0.973829,0.963409


## Generate Heatmaps

In [845]:
baseline_lstm.apply_heatmap()
guided_lstm.apply_heatmap()
baseline_gru.apply_heatmap()
guided_gru.apply_heatmap()

/Users/davidrau/miniconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
